# BERT-Based Models Training

In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from transformers import get_linear_schedule_with_warmup

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from sklearn.metrics import f1_score
import random


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Apr 15 15:33:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           On  | 00000000:2F:00.0 Off |                    0 |
| N/A   31C    P0              24W / 250W |      4MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Define Base Model

In [3]:
base_model = "bert-base-uncased"

### Datasets:

#### Moral Foundations Twitter Corpus:
The data can be downloaded here: https://osf.io/k5n7y/
Be aware that we have cleaned and pre-proceesed the corpus before using it therefore, the version we use might be sligthly different in numbers.

In [4]:
mftc_df = pd.read_csv('../Sm_Data/MFTC_tweets_MFT_10.csv')

In [5]:
# Assign a domain:
mftc_df['domain'] = 0

In [7]:
mftc_df.head()

,text,cleaned_text,annotations,new_label,subdomain,care,harm,fairness,cheating,loyalty,betrayal,authority,subversion,purity,degradation,domain
0,The courage to be impatient with evil and pati...,The courage to be impatient with evil and pati...,"[{'annotation': 'fairness', 'annotator': 'anno...",fairness,BLM,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,#NotAllCops but OMFG already. 😡 Protect and se...,but OMFG already. enraged_face Protect and ser...,"[{'annotation': 'care', 'annotator': 'annotato...",harm,BLM,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,"stop shaving, it's your manly dignity #blackje...","stop shaving, it's your manly dignity","[{'annotation': 'nm', 'annotator': 'annotator0...",nm,BLM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,ARABS MORTAL HATRED AND ENSLAVEMENT OF THE BLA...,ARABS MORTAL HATRED AND ENSLAVEMENT OF THE BLA...,"[{'annotation': 'harm,cheating', 'annotator': ...",harm,BLM,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,“@Babbsgirl2: #SheriffDavidClarke is my hero! ...,@user: is my hero! @user True patriot,"[{'annotation': 'nm', 'annotator': 'annotator0...",nm,BLM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [8]:
mftc_df['non-moral'] = mftc_df['new_label'].apply(lambda x: 1 if x == 'nm' else 0)

In [9]:
mftc_df.columns

Index(['text', 'cleaned_text', 'annotations', 'new_label', 'subdomain', 'care',
       'harm', 'fairness', 'cheating', 'loyalty', 'betrayal', 'authority',
       'subversion', 'purity', 'degradation', 'domain', 'non-moral'],
      dtype='object')

In [10]:
mftc_df.shape

(20628, 17)

#### Moral Foundations Reddit Corpus:
It can be dowloaded here: https://huggingface.co/datasets/USC-MOLA-Lab/MFRC



In [11]:
mfrc_df = pd.read_csv('../Sm_Data/Data/MFRC/MFRC_posts_MFT_10.csv')

In [12]:
mfrc_df.shape

(17741, 24)

#### Let's also add the general domain:

In [13]:
mfrc_df['domain'] = 1

In [14]:
mfrc_df.head(5)

,text,cleaned_text,subdomain,bucket,annotation,final_annotation,care,fairness,loyalty,authority,...,degradation,equality,proportionality,thin morality,non-moral,inconclusive,vader_neg,vader_neu,vader_pos,domain
0,That particular part of the debate is especial...,That particular part of the debate is especial...,europe,French politics,"{'annotator03': {'annotation': 'Non-Moral', 'c...",inconclusive,0,0,0,0,...,0,0,0,0,0,1,0.169,0.725,0.106,1
1,"/r/france is pretty lively, with it's own ling...","/r/france is pretty lively, with it's own ling...",europe,French politics,"{'annotator03': {'annotation': 'Non-Moral', 'c...",Non-Moral,0,0,0,0,...,0,0,0,0,1,0,0.142,0.679,0.180,1
2,TBH Marion Le Pen would be better. Closet fasc...,TBH Marion Le Pen would be better. Closet fasc...,neoliberal,French politics,"{'annotator03': {'annotation': 'Non-Moral', 'c...",inconclusive,0,0,0,0,...,0,0,0,0,0,1,0.358,0.498,0.144,1
3,it really is a very unusual situation isn't it...,it really is a very unusual situation isn't it...,europe,French politics,"{'annotator03': {'annotation': 'Non-Moral', 'c...",Non-Moral,0,0,0,0,...,0,0,0,0,1,0,0.118,0.772,0.110,1
4,The Le Pen brand of conservatism and classical...,The Le Pen brand of conservatism and classical...,europe,French politics,"{'annotator03': {'annotation': 'Authority', 'c...",inconclusive,0,0,0,0,...,0,0,0,0,0,1,0.000,0.795,0.205,1


We remove inconclusive reddit posts and the ones annotated as thin morality as we do not have such category in our data

In [16]:
mfrc_df = mfrc_df[~mfrc_df['final_annotation'].isin(['inconclusive', 'Thin Morality'])]

In [17]:
mfrc_df['non-moral'] = mfrc_df['final_annotation'].apply(lambda x: 1 if x == 'Non-Moral' else 0)

In [18]:
mfrc_df.shape

(13995, 25)

In [19]:
category_counts = mfrc_df[['care','harm', 'fairness', 'cheating', 'loyalty', 
       'betrayal', 'authority','subversion', 'purity', 'degradation', 'domain', 'non-moral']].sum()
category_counts

care             737
harm            1014
fairness         623
cheating         841
loyalty          241
betrayal         188
authority        330
subversion       357
purity           100
degradation      187
domain         13995
non-moral       9843
dtype: int64

In [20]:
# pie_charts(category_counts, 'MFRC_Distribution')

#### Facebook Vaccination Data:
For the acces of MFT annotated facebook data, we contacted the authors of the paper:https://www.semanticscholar.org/reader/9a42e991de4b3a8ac8f8673f271aa845cabd2294

In [21]:
mffp_df = pd.read_csv('../Sm_Data/Data/MFFP/MFFP_posts_MFT_10.csv')

In [22]:
mffp_df['domain'] = 2

In [23]:
mffp_df.head()

,text,cleaned_text,comment_id,page_id,page,class,care,harm,fairness,cheating,...,betrayal,authority,subversion,purity,degradation,liberty,oppression,non-moral,subdomain,domain
0,I just contacted my rep. Julia Brown and reque...,I just contacted my rep. Julia Brown and reque...,1525407617778182_1525810737737870,1374879262831019,1,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,vaccination,2
1,"The thing is, myself, and many other people ar...","The thing is, myself, and many other people ar...",2563364117066915_2563717157031611,414643305272351,1,1,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,vaccination,2
2,Soooooo....how do they explain all the dead ki...,Soooooo....how do they explain all the dead ki...,2563364117066915_2563383687064958,414643305272351,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,vaccination,2
3,"Well writer, I’m going to use a bit of your so...","Well writer, I m going to use a bit of your so...",2563364117066915_2563432760393384,414643305272351,1,1,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,vaccination,2
4,"Well imo, F***k the AVN! I am proud to say my ...","Well imo, F***k the AVN! I am proud to say my ...",10152270557898588_31662936,143367983587,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,vaccination,2


In [24]:
category_counts = mffp_df[['care','harm', 'fairness', 'cheating', 'loyalty', 
       'betrayal', 'authority','subversion', 'purity', 'degradation', 'domain', 'non-moral']].sum()
category_counts

care            357.0
harm            132.0
fairness        174.0
cheating        123.0
loyalty          40.0
betrayal         38.0
authority       110.0
subversion      204.0
purity           80.0
degradation     112.0
domain         3020.0
non-moral       248.0
dtype: float64

#### GPT-4 Synthtetic Lyrics Dataset

In [27]:
import re

def clean(lyrics):
  lyrics = re.sub("\(.*?\)", "", lyrics)
  lyrics = re.sub("\*", "", lyrics)
  lyrics = re.sub("\n", " ", lyrics)

  return lyrics

In [28]:
generated_lyrics_file = "../Lyrics_Data/Generated_Lyrics/GPT4_generated_songs_with_MFT_values.csv"

In [29]:
synth_lyrics = pd.read_csv(generated_lyrics_file)

In [30]:
synth_lyrics["subdomain"] = "synthetic_lyrics"
synth_lyrics["cleaned_text"] = synth_lyrics["lyrics"].apply(clean)

In [31]:
print(synth_lyrics.shape)
synth_lyrics.head()

(2721, 16)


,lyrics,genre,in_style_of_artist,Care,Harm,Fairness,Cheating,Loyalty,Betrayal,Authority,Subversion,Purity,Degradation,Moral_Combinations,subdomain,cleaned_text
0,"(Verse 1)\nThrough the shadows, I see the tear...",Rock,Last in Line,1,0,0,0,0,0,0,0,0,0,"('Care',)",synthetic_lyrics,"Through the shadows, I see the tears, Falling..."
1,(Verse 1)\nIn the alleyways and under the city...,Rock,The Walkmen,1,0,0,0,0,0,0,0,0,0,"('Care',)",synthetic_lyrics,"In the alleyways and under the city lights, F..."
2,"(Verse 1) \nIn twilight's silent whisper, whe...",Rock,Steven Wilson,1,0,0,0,0,0,0,0,0,0,"('Care',)",synthetic_lyrics,"In twilight's silent whisper, where shadows..."
3,"(Verse 1) \nIn the shattered glass of night, ...",Pop,Kandi,1,0,0,0,0,0,0,0,0,0,"('Care',)",synthetic_lyrics,"In the shattered glass of night, there's a ..."
4,"(Verse 1)\nIn the shadows, a whisper in the br...",Pop,The Pretty Reckless,1,0,0,0,0,0,0,0,0,0,"('Care',)",synthetic_lyrics,"In the shadows, a whisper in the breeze, A st..."


#### Concatenate the datasets together based on 5 Moral Values: Care, Fairness, Loyalty Authority and Purity:

In [32]:
cat_columns = mffp_df.columns[6:-5]

In [33]:
cat_columns

Index(['care', 'harm', 'fairness', 'cheating', 'loyalty', 'betrayal',
       'authority', 'subversion', 'purity', 'degradation'],
      dtype='object')

In [34]:
cat_columns = ["cleaned_text"]+cat_columns.tolist()+["subdomain", "domain"]

In [35]:
try:
  df = pd.concat([mftc_df[cat_columns], mfrc_df[cat_columns], mffp_df[cat_columns], emdf_df[cat_columns]], axis=0)
except:
  df = pd.concat([mftc_df[cat_columns], mfrc_df[cat_columns], mffp_df[cat_columns]], axis=0)

In [36]:
df = df.dropna(subset = ['cleaned_text'])
df = df[~(df[['care', 'fairness', 'loyalty', 'authority', 'purity', "harm", "subversion", "degradation", "cheating", "betrayal"]] == 2).any(axis=1)]
df.reset_index(drop = True, inplace = True)

In [37]:
df

,cleaned_text,care,harm,fairness,cheating,loyalty,betrayal,authority,subversion,purity,degradation,subdomain,domain
0,The courage to be impatient with evil and pati...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLM,0
1,but OMFG already. enraged_face Protect and ser...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLM,0
2,"stop shaving, it's your manly dignity",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLM,0
3,ARABS MORTAL HATRED AND ENSLAVEMENT OF THE BLA...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLM,0
4,@user: is my hero! @user True patriot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLM,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35882,Sadly most Canadians wont ever research vaccin...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,vaccination,2
35883,Stop poisoning the children,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,vaccination,2
35884,Was pregnant and ended up with the flu back in...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,vaccination,2
35885,My mother developed shingles last year and she...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,vaccination,2


In [38]:
df.shape

(35887, 13)

#### Neutral posts:

In [42]:
df[(df['care']==0) & (df['harm']==0) 
   & (df['fairness']==0) & (df['cheating']==0) 
   & (df['loyalty']==0)  & (df['betrayal']==0)
   & (df['authority']==0)& (df['subversion']==0)  
   & (df['purity']==0) & (df['degradation']==0)]

,cleaned_text,care,harm,fairness,cheating,loyalty,betrayal,authority,subversion,purity,degradation,subdomain,domain
2,"stop shaving, it's your manly dignity",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLM,0
4,@user: is my hero! @user True patriot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLM,0
8,@user_T_O_P_TERROR @user GLOBALIST COWARD PUPPETS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLM,0
11,The Israelites: Righteousness,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLM,0
13,"no tolerance 4 ple who come across righteous, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLM,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35871,They will be making them wear black uniforms n...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,vaccination,2
35876,I think I am more likely to have a bloody brai...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,vaccination,2
35877,Mandatory vaccination violates medical ethic o...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,vaccination,2
35880,Remember this - this can happen again if Non v...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,vaccination,2


##### Test Lyrics
When you have the test lyrics include them in the main dataset (later they're going to be included as test set only)

In [43]:
# use_test_lyrics = False

# try:
test_lyrics = pd.read_csv("../Lyrics_Data/MFT_human_annotated_lyrics.csv")  # change name to whatever name you used

test_lyrics =  test_lyrics.iloc[:,-12:]

test_lyrics.rename({k:k.split("_")[-1] for k in test_lyrics.columns[2:]}, inplace=True, axis=1)

test_lyrics.rename({"lyrics":"cleaned_text"}, inplace=True, axis=1)

test_lyrics["domain"] = len(pd.unique(df.domain))

test_lyrics["subdomain"] = "song_lyrics"

df = pd.concat([df, test_lyrics[cat_columns]], axis=0)

use_test_lyrics = True

# except:
#     print("No test lyrics yet! If you have input the test lyrics but you get this error, investigate further by removing the try/except statements!")

In [44]:
test_lyrics

,cleaned_text,Annotators,care,harm,fairness,cheating,loyalty,betrayal,authority,subversion,purity,degradation,domain,subdomain
0,The breeze with snow and mistletoe\nThe presen...,annotator_x,1,0,0,0,0,0,0,0,1,0,3,song_lyrics
1,I move across the earth\nDark shadows call my ...,annotator_x,0,1,0,0,0,0,0,1,0,0,3,song_lyrics
2,He came speeding through the town each day at ...,annotator_x,0,1,0,0,0,0,0,0,0,0,3,song_lyrics
3,"Mr., Mr., Mr., do you hear me?\nMr. Wankerman,...",annotator_x,0,0,0,0,0,1,0,1,0,0,3,song_lyrics
4,"Open your heart, open your mind\nA train is le...",annotator_x,1,0,0,0,0,0,0,0,0,0,3,song_lyrics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Tell me with affection in your voice\nThat you...,annotator_y,0,1,0,1,0,0,0,0,0,0,3,song_lyrics
196,"Well my woman, she showed up\nWith your number...",annotator_y,0,1,0,0,0,1,1,0,0,0,3,song_lyrics
197,Feels like I'm covered in lies\nso turn off th...,annotator_y,0,0,0,1,0,0,0,0,0,0,3,song_lyrics
198,You said you'd give me love\nInstead you cause...,annotator_y,0,0,0,1,0,1,0,0,0,0,3,song_lyrics


#### Synthetic Lyrics DF

In [45]:
synth_lyrics.columns = [c.lower() for c in synth_lyrics.columns]

In [46]:
synth_lyrics["domain"] = len(pd.unique(df.domain))#-1
synth_lyrics = synth_lyrics[cat_columns]

In [47]:
synth_lyrics.head()

,cleaned_text,care,harm,fairness,cheating,loyalty,betrayal,authority,subversion,purity,degradation,subdomain,domain
0,"Through the shadows, I see the tears, Falling...",1,0,0,0,0,0,0,0,0,0,synthetic_lyrics,4
1,"In the alleyways and under the city lights, F...",1,0,0,0,0,0,0,0,0,0,synthetic_lyrics,4
2,"In twilight's silent whisper, where shadows...",1,0,0,0,0,0,0,0,0,0,synthetic_lyrics,4
3,"In the shattered glass of night, there's a ...",1,0,0,0,0,0,0,0,0,0,synthetic_lyrics,4
4,"In the shadows, a whisper in the breeze, A st...",1,0,0,0,0,0,0,0,0,0,synthetic_lyrics,4


#### Suffle the data

In [51]:
synth_lyrics = synth_lyrics.sample(frac = 1)

In [52]:
synth_lyrics.reset_index(drop = True, inplace = True)

In [56]:
synth_lyrics.shape

(2721, 13)

In [53]:
df = df.sample(frac=1).reset_index(drop=True)

In [54]:
df.shape

(36087, 13)

In [57]:
df.isna().sum()

cleaned_text    0
care            0
harm            0
fairness        0
cheating        0
loyalty         0
betrayal        0
authority       0
subversion      0
purity          0
degradation     0
subdomain       0
domain          0
dtype: int64

In [58]:
synth_lyrics.isna().sum()

cleaned_text    0
care            0
harm            0
fairness        0
cheating        0
loyalty         0
betrayal        0
authority       0
subversion      0
purity          0
degradation     0
subdomain       0
domain          0
dtype: int64

In [59]:
df.domain.value_counts()

domain
0    20382
1    13995
2     1510
3      200
Name: count, dtype: int64

#### Text and label values:

In [61]:
text = df.cleaned_text.values
labels = df.iloc[:, 1:-2].values # 10 moral dimensions
labels2 = df.domain.values

In [62]:
df.iloc[:, 1:-2]

,care,harm,fairness,cheating,loyalty,betrayal,authority,subversion,purity,degradation
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
36082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
36083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
36084,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
36085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Text and labels values for synthetic lyrics

In [66]:
lyrics_text = synth_lyrics.cleaned_text.values
lyrics_labels = synth_lyrics.iloc[:, 1:-2].values

In [67]:
synth_lyrics.cleaned_text

0          Under the dimming lights of our once bright...
1        In the gentle hands of the dawn, a whispered ...
2        In the shadow of the night, under the fading ...
3        Under the glow of a blood moon's light, We sw...
4          Down by the old winding river, where the co...
                              ...                        
2716     In a town too small for secret whispers, Bene...
2717       In the heart of the city, beneath the neon ...
2718     Out here in this concrete jungle, we hustle, ...
2719     In the heart of the night, under a moonless s...
2720       Woke up with a stain on my heart, it's heav...
Name: cleaned_text, Length: 2721, dtype: object

In [68]:
synth_lyrics.iloc[:, 1:-2]

,care,harm,fairness,cheating,loyalty,betrayal,authority,subversion,purity,degradation
0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,1,0,0,0
2,0,1,0,1,0,0,0,0,0,0
3,0,1,0,0,0,1,0,0,1,0
4,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2716,0,0,0,0,0,1,0,1,0,0
2717,0,1,0,0,0,0,0,0,0,0
2718,1,0,0,0,0,0,0,0,0,0
2719,0,0,0,0,0,1,0,0,0,1


### Tokenization:

In [70]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModel.from_pretrained(base_model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [71]:
original_input_id = []
original_attention_masks = []
original_token_type_id = []

input_id = []
attention_masks = []
token_type_id = []

def preprocessing(input_text, tokenizer):
    '''
    Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
    '''
    return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 150,
                        padding = 'max_length',
                        return_attention_mask = True,
                        return_token_type_ids = True, 
                        return_tensors = 'pt',
                        truncation=True
                   )

for sample in text:
    # Original Input
    original_encoding_dict = preprocessing(sample, tokenizer)
    original_input_id.append(original_encoding_dict['input_ids'])
    original_attention_masks.append(original_encoding_dict['attention_mask'])


    # Calculate token type ids
    original_token_type = torch.zeros_like(original_encoding_dict['input_ids'])
    original_token_type[original_encoding_dict['input_ids'] != 0] = 0
    original_token_type_id.append(original_token_type)

original_input_id = torch.cat(original_input_id, dim=0)
original_attention_masks = torch.cat(original_attention_masks, dim=0)
original_token_type_id = torch.cat(original_token_type_id, dim = 0)
labels = torch.tensor(labels)  # add a new axis at index 1
labels2 = torch.tensor(labels2)  # add a new axis at index 1

In [72]:
# print('attention_masks.shape ', attention_masks.shape)
print('original_attention_masks.shape ', original_attention_masks.shape)
# print('input_id.shape ', input_id.shape)
print('original_input_id.shape ', original_input_id.shape)
# print('token_type_id.shape ', token_type_id.shape)
print('original_token_type_id.shape ', original_token_type_id.shape)

original_attention_masks.shape  torch.Size([36087, 150])
original_input_id.shape  torch.Size([36087, 150])
original_token_type_id.shape  torch.Size([36087, 150])


### Synthetic Lyrics Dataset tokenization

In [73]:
lyrics_input_id = []
lyrics_attention_masks = []
lyrics_token_type_id = []

input_id = []
attention_masks = []
token_type_id = []

def preprocessing(input_text, tokenizer):
    '''
    Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
    '''
    return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 300,
                        padding = 'max_length',
                        return_attention_mask = True,
                        return_token_type_ids = True,  # Add this line
                        return_tensors = 'pt',
                        truncation=True
                   )

for sample in lyrics_text:
    lyrics_encoding_dict = preprocessing(sample, tokenizer)
    lyrics_input_id.append(lyrics_encoding_dict['input_ids'])
    lyrics_attention_masks.append(lyrics_encoding_dict['attention_mask'])
   

    
    lyrics_token_type = torch.zeros_like(lyrics_encoding_dict['input_ids'])
    lyrics_token_type[lyrics_encoding_dict['input_ids'] != 0] = 0
    lyrics_token_type_id.append(lyrics_token_type)

lyrics_input_id = torch.cat(lyrics_input_id, dim=0)
lyrics_attention_masks = torch.cat(lyrics_attention_masks, dim=0)
lyrics_token_type_id = torch.cat(lyrics_token_type_id, dim = 0)
lyrics_labels = torch.tensor(lyrics_labels)  # add a new axis at index 1

In [74]:
print('lyrics_attention_masks.shape ', lyrics_attention_masks.shape)
print('lyrics_input_id.shape ', lyrics_input_id.shape)
print('lyrics_token_type_id.shape ', lyrics_token_type_id.shape)

lyrics_attention_masks.shape  torch.Size([2721, 300])
lyrics_input_id.shape  torch.Size([2721, 300])
lyrics_token_type_id.shape  torch.Size([2721, 300])


#### Encode corups for original embeddings

In [75]:
from transformers import BertModel

def encode_corpus(inputs, attentions, model, batch_size=16):
    all_embeddings = []
    for start_index in range(0, len(inputs), batch_size):
        b_input_ids = inputs[start_index:start_index+batch_size].to(model.device)
        b_input_mask = attentions[start_index:start_index+batch_size].to(model.device)
        with torch.no_grad():
            output = model(b_input_ids,
                      token_type_ids = None,
                      attention_mask = b_input_mask).last_hidden_state[:,0,:].detach().cpu()
            all_embeddings.extend(output)
    return torch.stack(all_embeddings)

In [76]:
print(labels.shape)
print(labels2.shape)

torch.Size([36087, 10])
torch.Size([36087])


In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Doman Adversarial Module:

In [78]:
from torch.autograd import Function
import torch.nn as nn

class GradientReversal(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.save_for_backward(x, alpha)
        return x

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = None
        _, alpha = ctx.saved_tensors
        if ctx.needs_input_grad[0]:
            grad_input = - alpha*grad_output
        return grad_input, None
revgrad = GradientReversal.apply

class GradientReversal(nn.Module):
    def __init__(self, alpha):
        super().__init__()
        self.alpha = torch.tensor(alpha, requires_grad=False)

    def forward(self, x):
        return revgrad(x, self.alpha)

class AdversarialBERT(nn.Module):
    def __init__(self, bert_model, moral_label=5, domain_label=2, class_weight=[1,1],
                 domain_weight=1, identity_weight=1, reconstruction_weight=1, moral_weight=1,
                 alpha=1.0,
                 freeze_bert=False): # class_weight[0] = -1 deactivates the baalancing tentatives

        super(AdversarialBERT, self).__init__()
        self.bert = bert_model
        bert_dim = 768
        self.invariant_trans = nn.Linear(768, 768)
        print(' self.invariant_trans ',  self.invariant_trans)
        if identity_weight+reconstruction_weight+domain_weight==0:
            self.moral_classification = nn.Linear(768, moral_label)
        else:
            self.moral_classification = nn.Sequential(nn.Linear(768,768),
                                                      nn.ReLU(),
                                                      nn.Linear(768, moral_label))

        self.domain_classification = nn.Sequential(GradientReversal(alpha),
                                                   nn.Linear(768,768),
                                                   nn.ReLU(),
                                                   nn.Dropout(0.3),
                                                   nn.Linear(768, domain_label))

        # Dynamically adjustable alpha for gradient reversal
        self.alpha = alpha
        self.domain_weight = domain_weight

        if moral_label>2:
                self.loss_fn_moral = nn.BCEWithLogitsLoss() 

        else:
            if class_weight[0]>0:
                weights = torch.tensor(class_weight).float()
            else:
                weights = torch.tensor([1.0 for _ in range(moral_label)]).float()
            if moral_label>2:
                self.loss_fn_moral = nn.BCEWithLogitsLoss(pos_weight=weights) 

            else:
                self.loss_fn_moral = nn.CrossEntropyLoss(weight=weights)

        self.loss_fn_domain = nn.CrossEntropyLoss()
        self.reconstruction_feed = nn.Linear(768, 768)
        self.loss_reconstruction = nn.MSELoss()
        self.weight_identity = identity_weight
        self.reconstruction_weight = reconstruction_weight
        self.moral_weight = moral_weight
        self.identity = torch.eye(768).to(device)
        self.freeze=freeze_bert

    def update_alpha(self, new_alpha):
        # Method to update alpha for the gradient reversal layer
        self.domain_classification[0].alpha = new_alpha

    def update_model_params(self, domain_weight, moral_weight, reconstruction_weight, identity_weight):
        # Update the relevant parameters
        self.domain_weight = domain_weight
        self.moral_weight = moral_weight
        self.reconstruction_weight = reconstruction_weight
        self.identity_weight = identity_weight

    def forward(self, b_input_ids, b_token_type_ids, b_input_mask, b_labels, b_domain_labels, original_bert_embeddings=None, test=False):
        # Forward pass
        if self.freeze:
            with torch.no_grad():
                pooled_output = self.bert(b_input_ids,
                                    token_type_ids = b_token_type_ids, #it was None
                                    attention_mask = b_input_mask).last_hidden_state[:,0,:]

        else:
            pooled_output = self.bert(b_input_ids,
                                token_type_ids = b_token_type_ids, #it was None
                                attention_mask = b_input_mask).last_hidden_state[:,0,:]


        pooled_output = self.invariant_trans(pooled_output)

        logits = self.moral_classification(pooled_output)

        loss_moral = self.loss_fn_moral(logits, b_labels)

        if test:
            return loss_moral, logits
        if self.domain_weight>0:
            loss_domain = self.loss_fn_domain(self.domain_classification(pooled_output), b_domain_labels)
        else:
            loss_domain=0
        if original_bert_embeddings is not None:
            loss_reconstruction = self.loss_reconstruction(self.reconstruction_feed(pooled_output), original_bert_embeddings)*self.reconstruction_weight
        else:
            loss_reconstruction=0
        if self.weight_identity>0:
            loss_identity = torch.norm(self.invariant_trans.weight-self.identity)*self.weight_identity
        else:
            loss_identity=0
        total_loss = loss_moral*self.moral_weight+loss_reconstruction+loss_identity+self.domain_weight*loss_domain
        return  total_loss

# Single Label Experiments

In [81]:
from sklearn.model_selection import train_test_split

batch_size = 16

def adjust_domain_label(labels, test_domain):
    return torch.tensor([l if l < test_domain else l - 1 for l in labels.tolist()])

add_lyrics_training = True  # True to use the synthetic lyrics in training, False to train the models with only social media data 
just_lyrics = False  # Change this to True if you want to train just on the synthetic lyrics

count_synthetic_in_model_weights = True  # This option involves the counting of the synthetic labels in computing class weights. 

if just_lyrics:
    add_lyrics_training = True
    print("The model is trained on just Synthetic Lyrics")

if add_lyrics_training:
    print("The model is trained with Social Media Data and Synthetic Lyrics")

test_domain = True

if use_test_lyrics:
    test_domain = 3
    train_idx = df[df.domain != test_domain].index
    val_idx = df[df.domain == test_domain].index
elif test_domain:
    test_domain = 2
    train_idx = df[df.domain != test_domain].index
    val_idx = df[df.domain == test_domain].index
else:
    train_idx, test_idx = train_test_split(df.index, test_size=0.2, random_state=42)

if just_lyrics:
    suffix = "_with_just_synthetic_lyrics"
    print(suffix)  
elif add_lyrics_training:
    suffix = "_with_social_media_and_synthetic_lyrics"
    print(suffix)  
else:
    suffix = "_with_social_media"
    print(suffix)   

The model is trained with Social Media Data and Synthetic Lyrics
_with_social_media_and_synthetic_lyrics


### Training:

In [85]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import multilabel_confusion_matrix as mcm, classification_report
from itertools import product
from transformers import get_linear_schedule_with_warmup
import torch
from tqdm.auto import trange
import numpy as np

import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

possible_labels = ["care", "harm", "fairness", "cheating", "loyalty", "betrayal",
                   "authority", "subversion", "purity", "degradation"]

bert_original_embeddings = None

for lab_idx, lab in enumerate(possible_labels):
    best_f1 = 0
    num_labels = 2
    num_domains = 4

    print(f"Number of Domains: {num_domains}")
    
    epochs = 20
    batch_size = 16
    epoch_count = 0
    rw = 0.1
    iw = 0.01
    dw = 0.1

    initial_alpha = 0.1
    alpha_growth_rate = 0.1

    print(f'Models for predicting 10 Moral Dimensions')
    print(f'Parameters: reconstruction_weight = {rw}; identity_weight = {iw}; domain_weight = {dw}.')

    bert_model = AutoModel.from_pretrained(base_model).cuda()

    if bert_original_embeddings is None:
        bert_original_embeddings = encode_corpus(original_input_id[train_idx],
                                                 original_attention_masks[train_idx],
                                                 bert_model)
        if add_lyrics_training:
            lyrics_bert_original_embeddings = encode_corpus(lyrics_input_id,
                                                            lyrics_attention_masks,
                                                            bert_model)
    bert_model = bert_model.to("cuda")
    new_labels = []
    for ex in labels[train_idx]:
        if ex[lab_idx]:
            new_labels.append(1)
        else:
            new_labels.append(0)

    if just_lyrics:
        pos_weight = [0, 0]
    else:
        pos_weight = [sum(new_labels), len(new_labels)]

    train_set = TensorDataset(original_input_id[train_idx],
                              original_token_type_id[train_idx],
                              original_attention_masks[train_idx],
                              torch.tensor(new_labels),
                              adjust_domain_label(labels2[train_idx], test_domain),
                              bert_original_embeddings)
    new_labels = []
    for ex in labels[val_idx]:
        if ex[lab_idx]:
            new_labels.append(1)
        else:
            new_labels.append(0)

    val_set = TensorDataset(original_input_id[val_idx],
                            original_token_type_id[val_idx],
                            original_attention_masks[val_idx],
                            torch.tensor(new_labels),
                            labels2[val_idx])

    if add_lyrics_training:
        new_labels = []
        for ex in lyrics_labels:
            if ex[lab_idx]:
                new_labels.append(1)
            else:
                new_labels.append(0)

        if count_synthetic_in_model_weights:
            pos_weight = [pos_weight[0]+sum(new_labels), pos_weight[1]+len(new_labels)]

        if use_test_lyrics:
            domain_labels = torch.tensor([test_domain for _ in range(len(lyrics_labels))])
        else:
            domain_labels = torch.tensor([len(pd.unique(df)) for _ in range(len(lyrics_labels))])

        lyrics_train_set = TensorDataset(lyrics_input_id,
                                         lyrics_token_type_id,
                                         lyrics_attention_masks,
                                         torch.tensor(new_labels),
                                         domain_labels,
                                         lyrics_bert_original_embeddings)

    pos_weight = pos_weight[0] / pos_weight[1]
    class_weight = [pos_weight, 1 - pos_weight]

    train_dataloader = DataLoader(train_set,
                                  sampler=RandomSampler(train_set),
                                  batch_size=batch_size)

    if add_lyrics_training:
        train_dataloader_lyrics = DataLoader(lyrics_train_set,
                                             sampler=RandomSampler(lyrics_train_set),
                                             batch_size=batch_size // 4)

    validation_dataloader = DataLoader(val_set,
                                       batch_size=batch_size)

    total_steps = len(train_dataloader) * epochs

    def pick_the_model(adversarial, model, class_weight, epoch):
        print('Adversarial', adversarial)
        if adversarial:
            model.update_model_params(domain_weight=dw, moral_weight=1,
                                      reconstruction_weight=rw, identity_weight=iw)
        else:
            model.update_model_params(domain_weight=0, moral_weight=1,
                                      reconstruction_weight=0, identity_weight=0)
        model = model.to(device)
        return model

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = AdversarialBERT(bert_model, moral_label=num_labels,
                            domain_label=num_domains, domain_weight=dw, moral_weight=1,
                            reconstruction_weight=rw, identity_weight=iw,
                            alpha=initial_alpha, class_weight=class_weight,
                            freeze_bert=False).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    for epoch in trange(epochs, desc='Epoch'):
        print('Epoch: ', epoch)
        adversarial = False if just_lyrics else True

        model = pick_the_model(adversarial, model, class_weight, epoch)
        model.train()
        current_alpha = torch.tensor(initial_alpha + alpha_growth_rate * epoch, requires_grad=False).to(device)
        model.update_alpha(current_alpha)

        tr_loss, nb_tr_examples, nb_tr_steps = 0, 0, 0

        if not just_lyrics:
            for step, batch in enumerate(train_dataloader):
                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_token_type_ids, b_input_mask, b_labels, b_domain_labels, bert_embeddings = batch
                optimizer.zero_grad()
                loss = model(b_input_ids, b_token_type_ids, b_input_mask, b_labels, b_domain_labels, bert_embeddings)
                loss.backward()
                optimizer.step()
                scheduler.step()

                tr_loss += loss.item()
                nb_tr_examples += b_input_ids.size(0)
                nb_tr_steps += 1

        if add_lyrics_training:
            for step, batch in enumerate(train_dataloader_lyrics):
                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_token_type_ids, b_input_mask, b_labels, b_domain_labels, bert_embeddings = batch
                optimizer.zero_grad()
                loss = model(b_input_ids, b_token_type_ids, b_input_mask, b_labels, b_domain_labels, bert_embeddings)
                loss.backward()
                optimizer.step()
                scheduler.step()

                tr_loss += loss.item()
                nb_tr_examples += b_input_ids.size(0)
                nb_tr_steps += 1

        model.eval()
        val_loss, nb_val_examples, nb_val_steps = 0, 0, 0
        y_true, y_pred = [], []

        for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_token_type_ids, b_input_mask, b_labels, b_domain_labels = batch

            with torch.no_grad():
                loss, logits = model(b_input_ids, b_token_type_ids, b_input_mask, b_labels, b_domain_labels, test=True)

                val_loss += loss.item()
                logits = logits.detach().cpu().numpy()
                nb_val_examples += b_input_ids.size(0)
                nb_val_steps += 1
                label_ids = b_labels.to('cpu').numpy()
                predicted_labels = np.argmax(logits, axis=1)
                y_true.extend(label_ids)
                y_pred.extend(predicted_labels)

        epoch_count += 1
        print('Evaluation')

        target_names = [f"Non-{lab}", lab]
        report = classification_report(y_true, y_pred, target_names=target_names)
        f1 = f1_score(y_true, y_pred, average="binary")
        print('F1 score', f1)
        print("\nClassification Report:")
        print(report)

        if f1 > best_f1:
            best_f1 = f1
            print('best_F1', best_f1)
            print("We are saving the state of this model")
            torch.save(model.state_dict(), f"../Models/moralBERT_sm_sl/model_bert_{lab}{suffix}.bin")

        try:
            print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
            print('\t - Validation loss: {:.4f}'.format(val_loss / nb_val_steps))

        except ZeroDivisionError:
            print("No predicted positives...")
